In [1]:
# !pip install ultralytics
import numpy as np
import cv2
import re
import skimage.io as io
import warnings
from tqdm import tqdm
from skimage import img_as_ubyte, filters
from scipy.ndimage import binary_fill_holes
from glob import glob
from skimage import morphology, measure
from yolo_segmentation import YOLOSegmentation
from PIL import Image, ImageFilter

In [2]:
warnings.filterwarnings("ignore")

In [27]:
def find_bighest_cluster_area(clusters):
    regions = measure.regionprops(clusters)

    all_areas = map(lambda item: item.area, regions)

    return max(all_areas)

def find_bighest_cluster(clusters):

    clusters = measure.label(clusters, background=0)

    cluster_size = find_bighest_cluster_area(clusters)
    
    cluster_size = 1 if cluster_size == 0 else cluster_size

    return morphology.remove_small_objects(clusters,
                                           min_size=(cluster_size - 1),
                                           connectivity=8)

def get_image_area(image):
    try:
        return find_bighest_cluster_area(measure.label(image, background=0))
    except:
        return 0
    
def try_all_models(image):
    
    response = None
    
    paths = [
        # 'yolov8x-seg.pt',
        # 'yolov8l-seg.pt',
        'yolov8m-seg.pt',
        'yolov8s-seg.pt',
        'yolov8n-seg.pt',
    ]
    
    for model_path in paths:
        
        model = YOLOSegmentation("models/" + model_path)
    
        bboxes, classes, segmentations, scores = model.detect(image)
        
        if bboxes == ():
            continue
        else:
            return bboxes, classes, segmentations, scores
        
    return (), (), (), ()

def build_mask(img):
    
    bboxes, classes, segmentations, scores = try_all_models(img)
    
    if bboxes == ():
        return

    x, y, _ = img.shape

    mask = np.zeros((x, y))

    for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):

        (x, y, x2, y2) = bbox

        cv2.polylines(mask, [seg], True, (255, 255, 255), 1)
        
        
    filled = binary_fill_holes(mask)

    return find_bighest_cluster(filled)

In [ ]:
for path in glob("../data/coletas/combined/images/*"):
    
    try:
        frame = io.imread(path)
        
        filename = path.split("/")[-1]
        
        mask = build_mask(frame)
    
        #     new_ = np.zeros(image.shape)
        #     msk = filters.median(frame, selem=np.ones((3, 3)))

        #     new_[:,:,0] = msk * image[:,:,0]
        #     new_[:,:,1] = msk * image[:,:,1]
        #     new_[:,:,2] = msk * image[:,:,2]

        io.imsave(f"output/{filename}", mask)
    
    except Exception as e:
        print("Erro ao processar [ ", path, end=" ]")
        continue

Ultralytics YOLOv8.0.43 🚀 Python-3.9.13 torch-1.13.1+cu117 CPU
YOLOv8x-seg summary (fused): 295 layers, 71797696 parameters, 0 gradients, 344.1 GFLOPs

0: 512x640 1 bird, 520.2ms
Speed: 0.6ms preprocess, 520.2ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Ultralytics YOLOv8.0.43 🚀 Python-3.9.13 torch-1.13.1+cu117 CPU
YOLOv8x-seg summary (fused): 295 layers, 71797696 parameters, 0 gradients, 344.1 GFLOPs

0: 512x640 1 surfboard, 609.7ms
Speed: 0.3ms preprocess, 609.7ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Ultralytics YOLOv8.0.43 🚀 Python-3.9.13 torch-1.13.1+cu117 CPU
YOLOv8x-seg summary (fused): 295 layers, 71797696 parameters, 0 gradients, 344.1 GFLOPs

0: 512x640 1 bird, 574.1ms
Speed: 0.3ms preprocess, 574